In [1]:
import sqlite3

import os

conn = sqlite3.connect('../../neurhome-testdata/application_log.db')
c = conn.cursor()

In [2]:
import collections

ApplicationRecord =  collections.namedtuple('ApplicationRecord', 'package,timestamp,geohash,wifi')
entries = [ ApplicationRecord._make(row) for row in conn.execute("select package,timestamp,geohash,wifi from application_log")]


In [3]:
import functools, collections

def acc_fn(acc, entry):
    acc[entry[0]].append(entry)
    return acc

packages = functools.reduce(acc_fn, entries, collections.defaultdict(list))


In [4]:
import datetime

def calculate_score_and_sort(position, network, time):
    print("using %s" % locals())
    
    [hour, minute] = map(int, time.split(":"))
    norm_now = hour * 60 + minute

    def score_entry(e):
        lcs = 0
        for (a,b) in zip(position, e.geohash):
            if a==b:
                lcs+=1
            else:
                break

        entry_time = datetime.datetime.fromisoformat(e.timestamp)
        entry_time_norm = entry_time.hour * 60 + entry_time.minute

        timeproximity = 1.0 - abs(entry_time_norm - norm_now) % 720.0 / 720

        return (int(e.wifi == network)/2, lcs/12/2, timeproximity)

    def score_package(package_entries):
        return tuple(map(lambda x: x/len(package_entries), 
                   functools.reduce(lambda acc, scores: (a+b for a,b in zip(acc,scores)),
                                    map(score_entry, package_entries),
                                    (0,0,0))))+(len(package_entries)/len(entries),)
    
    result = [{
        'package': package,
        'component_score': score_package(entries), 
        'count': len(entries),
        'total_score': sum(score_package(entries))
    } for package, entries in packages.items()]
    result.sort(key=lambda e: e['total_score'] , reverse=True)
    return result


In [5]:
from IPython.display import HTML, display, update_display, clear_output
from jinja2 import Template
from ipywidgets import widgets, interact,TwoByTwoLayout

wifi_input = widgets.Text(description='wifi', value='VM494929B')
geohash_input = widgets.Text(description='geohash', value='gc7x3rx0w8qx')
time_input = widgets.Text(description='time', placeholder='hh:mm', value='23:30')
calculate_button = widgets.Button(description='calculate')

def on_button_clicked(b):
    template = Template('''
    <table>
        <tr>
            <th>package</th>
            <th>count</th>
            <th>total score</th>
            <th>wifi</th>
            <th>location</th>
            <th>time</th>
            <th>count proportion</th>
        </tr>
        {% for row in table %}
        <tr>
            <td>{{row['package']}}</td>
            <td>{{row['count']}}</td>
            <td>{{"%.2f" % row['total_score']}}</td>
            {% for score_item in row['component_score'] %}
            <td>{{"%.2f" % score_item}}</td>
            {% endfor %}
        </tr>
        {% endfor %}
    </table>
    ''')
    
    clear_output(wait=True)
    
    display(fields_layout, display_id='packages_table')
    
    result = calculate_score_and_sort(
        geohash_input.value,
        wifi_input.value,
        time_input.value
    )
    
    
    display(HTML(template.render(table=result[:20])), display_id='packages_table')

calculate_button.on_click(on_button_clicked)

fields_layout = TwoByTwoLayout(
    top_left=wifi_input,
                   top_right=geohash_input,
                   bottom_left=time_input,
                   bottom_right=calculate_button)

display(fields_layout, display_id='packages_table')


TwoByTwoLayout(children=(Text(value='VM494929B', description='wifi', layout=Layout(grid_area='top-left')), Tex…

using {'time': '23:30', 'network': 'VM494929B', 'position': 'gc7x3rx0w8qx'}


package,count,total score,wifi,location,time,count proportion
com.concur.breeze,2,1.83,0.50,0.35,0.98,0.00
com.andrewshu.android.redditdonation,278,1.71,0.46,0.34,0.55,0.36
com.asmodeedigital.tickettoride1journey,1,1.69,0.50,0.33,0.86,0.00
com.lastpass.lpandroid,3,1.66,0.50,0.38,0.78,0.00
com.android.chrome,156,1.60,0.46,0.34,0.59,0.20
appear.in.app,4,1.58,0.50,0.35,0.72,0.01
com.aidungeon,2,1.57,0.50,0.38,0.69,0.00
com.google.android.youtube,34,1.56,0.47,0.34,0.70,0.04
com.google.android.apps.fitness,2,1.52,0.50,0.31,0.70,0.00
com.kiarygames.tinyroom,1,1.51,0.50,0.38,0.64,0.00
